### Load Packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import re #Regular Expressions
%matplotlib inline

### Import Data

In [24]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [25]:
#Combine train & test
titanic = pd.concat([train,test], sort = False)
titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


### Treat Missing Values

In [26]:
#Check for missing values
titanic.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [27]:
#Treat Numericals
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].median())
titanic['Fare'] = titanic['Fare'].fillna(titanic['Fare'].median())

In [28]:
#Treat Embarked
#titanic['Embarked'].value_counts().idxmax()
titanic['Embarked'] = titanic['Embarked'].fillna(titanic['Embarked'].value_counts().idxmax())

In [29]:
#Treat Cabin
titanic['Cabin'] = titanic['Cabin'].fillna('Missing')

In [30]:
titanic['Cabin'] = titanic['Cabin'].str[0]
titanic['Cabin'].value_counts()

M    1014
C      94
B      65
D      46
E      41
A      22
F      21
G       5
T       1
Name: Cabin, dtype: int64

In [31]:
#Encode Sex 
titanic['Sex'] = titanic['Sex'].map({'male': 1, 'female': 0})
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null int64
Age            1309 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          1309 non-null object
Embarked       1309 non-null object
dtypes: float64(3), int64(5), object(4)
memory usage: 132.9+ KB


### New Features

#### Create Family Size and IsAlone

In [18]:
titanic['FamilySize'] = titanic['SibSp'] + titanic['Parch'] + 1

titanic['IsAlone'] = 0
titanic.loc[titanic['FamilySize'] == 1, 'IsAlone'] = 1

In [33]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    1309 non-null int64
Survived       891 non-null float64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null int64
Age            1309 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          1309 non-null object
Embarked       1309 non-null object
dtypes: float64(3), int64(5), object(4)
memory usage: 132.9+ KB


#### Extract Title

In [35]:
#Extract Title
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

In [36]:
titanic['Title'] = titanic['Name'].apply(get_title)

In [38]:
titanic.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.25,M,S,Mr


In [39]:
#Reduce Categories
titanic['Title'] = titanic['Title'].replace(['Capt', 'Col','Dr', 'Major', 'Rev'], 'Officer')
titanic['Title'] = titanic['Title'].replace(['Lady', 'Countess','Don', 'Sir', 'Jonkheer', 'Dona'], 'Royalty')
titanic['Title'] = titanic['Title'].replace(['Mlle', 'Ms'], 'Miss')
titanic['Title'] = titanic['Title'].replace('Mme', 'Mrs')

In [40]:
titanic['Title'].value_counts()

Mr         757
Miss       264
Mrs        198
Master      61
Officer     23
Royalty      6
Name: Title, dtype: int64

#### Extract Ticket

In [42]:
#Extract Ticket
def get_ticket(ticket):
    txt = ticket.replace("/","")
    txt = txt.replace(".","")
    txt = txt.upper()
    
    if txt.isdigit():
        return 'xxx'
    else:
        title_search = re.search(r'[A-Z]+[A-Z0-9]+|[A-Z]|[A-Za-z]', txt)
        # If the title exists, extract and return it.
        if title_search:
            return title_search.group(0)
        return ""

In [43]:
titanic['Ticket'] = titanic['Ticket'].apply(get_ticket)

In [44]:
titanic['Ticket'].value_counts()

xxx        957
PC          92
CA          68
A5          28
SOTONOQ     24
SCPARIS     19
WC          15
STONO       14
A4          10
FCC          9
C            8
SOC          8
SOPP         7
STONO2       7
SCAH         5
WEP          4
LINE         4
PP           4
SOTONO2      3
FC           3
PPP          2
SC           2
SCA4         2
SWPP         2
AQ3          1
AS           1
CASOTON      1
SCA3         1
A            1
AQ4          1
LP           1
STONOQ       1
SOP          1
SCOW         1
FA           1
SP           1
Name: Ticket, dtype: int64

### Prepare Data for Modelling

In [45]:
titanic_all = titanic.drop(['PassengerId','Name','SibSp','Parch'],axis=1)

In [47]:
titanic_dummies.head()

,Survived,Pclass,Sex,Age,Fare,Ticket_A4,Ticket_A5,Ticket_AQ3,Ticket_AQ4,Ticket_AS,...,Cabin_G,Cabin_M,Cabin_T,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
0,0.0,3,1,22.0,7.2500,0,1,0,0,0,...,0,1,0,0,1,0,1,0,0,0
1,1.0,1,0,38.0,71.2833,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1.0,3,0,26.0,7.9250,0,0,0,0,0,...,0,1,0,0,1,1,0,0,0,0
3,1.0,1,0,35.0,53.1000,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,0.0,3,1,35.0,8.0500,0,0,0,0,0,...,0,1,0,0,1,0,1,0,0,0


In [46]:
titanic_dummies=pd.get_dummies(titanic_all, drop_first=True)
titanic_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 55 columns):
Survived          891 non-null float64
Pclass            1309 non-null int64
Sex               1309 non-null int64
Age               1309 non-null float64
Fare              1309 non-null float64
Ticket_A4         1309 non-null uint8
Ticket_A5         1309 non-null uint8
Ticket_AQ3        1309 non-null uint8
Ticket_AQ4        1309 non-null uint8
Ticket_AS         1309 non-null uint8
Ticket_C          1309 non-null uint8
Ticket_CA         1309 non-null uint8
Ticket_CASOTON    1309 non-null uint8
Ticket_FA         1309 non-null uint8
Ticket_FC         1309 non-null uint8
Ticket_FCC        1309 non-null uint8
Ticket_LINE       1309 non-null uint8
Ticket_LP         1309 non-null uint8
Ticket_PC         1309 non-null uint8
Ticket_PP         1309 non-null uint8
Ticket_PPP        1309 non-null uint8
Ticket_SC         1309 non-null uint8
Ticket_SCA3       1309 non-null uint8
Ticket_SCA4    

In [48]:
#Get Train
t_train = titanic_dummies[titanic_dummies['Survived'].notnull()]
t_train.describe()

,Survived,Pclass,Sex,Age,Fare,Ticket_A4,Ticket_A5,Ticket_AQ3,Ticket_AQ4,Ticket_AS,...,Cabin_G,Cabin_M,Cabin_T,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.0,891.0,891.000000,...,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,0.647587,29.361582,32.204208,0.007856,0.023569,0.0,0.0,0.001122,...,0.004489,0.771044,0.001122,0.086420,0.725028,0.207632,0.580247,0.141414,0.020202,0.005612
std,0.486592,0.836071,0.477990,13.019697,49.693429,0.088337,0.151787,0.0,0.0,0.033501,...,0.066890,0.420397,0.033501,0.281141,0.446751,0.405840,0.493796,0.348644,0.140770,0.074743
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,22.000000,7.910400,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,3.000000,1.000000,28.000000,14.454200,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,1.000000,3.000000,1.000000,35.000000,31.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,3.000000,1.000000,80.000000,512.329200,1.000000,1.000000,0.0,0.0,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [49]:
#Get Test
t_test = titanic_dummies[titanic_dummies['Survived'].isnull()]
t_test.describe()

,Survived,Pclass,Sex,Age,Fare,Ticket_A4,Ticket_A5,Ticket_AQ3,Ticket_AQ4,Ticket_AS,...,Cabin_G,Cabin_M,Cabin_T,Embarked_Q,Embarked_S,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
count,0.0,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.0,...,418.000000,418.000000,418.0,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,NaN,2.265550,0.636364,29.805024,35.576535,0.007177,0.016746,0.002392,0.002392,0.0,...,0.002392,0.782297,0.0,0.110048,0.645933,0.188995,0.574163,0.172249,0.011962,0.002392
std,NaN,0.841838,0.481622,12.667969,55.850103,0.084514,0.128474,0.048912,0.048912,0.0,...,0.048912,0.413179,0.0,0.313324,0.478803,0.391974,0.495062,0.378049,0.108844,0.048912
min,NaN,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,1.000000,0.000000,23.000000,7.895800,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,3.000000,1.000000,28.000000,14.454200,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,1.000000,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,NaN,3.000000,1.000000,35.750000,31.471875,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,1.000000,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,NaN,3.000000,1.000000,76.000000,512.329200,1.000000,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [51]:
X = t_train.drop('Survived', axis = 1)
y = t_train['Survived']

In [52]:
import sklearn.model_selection as model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=0.2, random_state = 200)

### Modelling

#### Logistic Regression (Kaggle : 0.77)

In [54]:
#Import libraries
from sklearn.linear_model import LogisticRegression

In [57]:
#Build model
logreg = LogisticRegression(solver='lbfgs', max_iter= 1000)
logreg.fit( X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [58]:
print(f'Train : {logreg.score(X_train, y_train):.2f}')
print(f'Test : {logreg.score(X_test, y_test):.2f}')

Train : 0.83
Test : 0.77


In [59]:
sub_test = t_test.drop(['Survived'], axis = 1)
sub_test_pred = logreg.predict(sub_test).astype(int)

In [60]:
AllSub = pd.DataFrame({ 'PassengerId': test['PassengerId'],
                       'Survived' : sub_test_pred
    
})

AllSub.to_csv("Basic_Log_model.csv", index = False)

#### Bagging Classifier (Kaggle : 0.69)

In [62]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [63]:
bclf=BaggingClassifier(oob_score=False,n_jobs=-1,n_estimators=20,random_state=200,
                      base_estimator=DecisionTreeClassifier())

In [64]:
from sklearn.model_selection import GridSearchCV

In [65]:
parameters = {'n_estimators' : (5,10,15,25,30,35,40,35,50)}

In [66]:
Bag_grid  = GridSearchCV(bclf, param_grid = parameters, cv = 3)

In [67]:
Bag_model = Bag_grid.fit(X_train, y_train)

In [68]:
Bag_grid.best_estimator_

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,


In [69]:
print(f'Train : {Bag_model.score(X_train, y_train):.2f}')
print(f'Test : {Bag_model.score(X_test, y_test):.2f}')

Train : 0.98
Test : 0.77


In [70]:
sub_test = t_test.drop(['Survived'], axis = 1)
sub_test_pred = Bag_model.predict(sub_test).astype(int)

In [71]:
AllSub = pd.DataFrame({ 'PassengerId': test['PassengerId'],
                       'Survived' : sub_test_pred
    
})

AllSub.to_csv("GRID_CV_Bagging_model.csv", index = False)

#### Random Forest

In [73]:
from sklearn.ensemble import RandomForestClassifier

In [72]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 300, num = 30)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 20)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [74]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 15, 20, 25, 31, 36, 41, 46, 52, 57, 62, 67, 73, 78, 83, 88, 94, 99, 104, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [75]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:   21.0s finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [76]:
rf_random.best_params_

{'n_estimators': 180,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 25,
 'bootstrap': True}

In [81]:
print(f'Train : {rf_random.score(X_train, y_train):.2f}')
print(f'Test : {rf_random.score(X_test, y_test):.2f}')

Train : 0.89
Test : 0.79


In [83]:
sub_test = t_test.drop(['Survived'], axis = 1)
sub_test_pred = rf_random.predict(sub_test).astype(int)

In [84]:
AllSub = pd.DataFrame({ 'PassengerId': test['PassengerId'],
                       'Survived' : sub_test_pred
    
})

AllSub.to_csv("Randonm_CV_RF_model.csv", index = False)